In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
dfTrainX = pd.read_csv('/kaggle/input/titanic/train.csv')
dfTrainY = dfTrainX['Survived']
dfTrainX = dfTrainX.drop(['Survived', 'Name', 'Cabin', 'Ticket'], axis = 1)
dfTrainX.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S


In [3]:
sexEncoder = pd.get_dummies(dfTrainX['Sex'])
dfTrainX = dfTrainX.drop('Sex', axis = 1)
dfTrainX = dfTrainX.join(sexEncoder)
embarkEncoder = pd.get_dummies(dfTrainX['Embarked'])
dfTrainX = dfTrainX.drop('Embarked', axis = 1)
dfTrainX = dfTrainX.join(embarkEncoder)
dfTrainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Age          714 non-null    float64
 3   SibSp        891 non-null    int64  
 4   Parch        891 non-null    int64  
 5   Fare         891 non-null    float64
 6   female       891 non-null    uint8  
 7   male         891 non-null    uint8  
 8   C            891 non-null    uint8  
 9   Q            891 non-null    uint8  
 10  S            891 non-null    uint8  
dtypes: float64(2), int64(4), uint8(5)
memory usage: 46.2 KB


In [4]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean') #could also use mean, median, most freq
dfTrainX = pd.DataFrame(imp.fit_transform(dfTrainX), columns = dfTrainX.columns)

In [5]:
logistic_regression = LogisticRegression(solver = 'liblinear', max_iter = 500)
logistic_regression.fit(dfTrainX, dfTrainY)
# logistic_regression.score(dfTestX, dfTestY)

LogisticRegression(max_iter=500, solver='liblinear')

In [6]:
titanic_in = pd.read_csv('/kaggle/input/titanic/test.csv')
titanic_Passengers = titanic_in['PassengerId']
titanic_in = titanic_in.drop(['Name', 'Cabin', 'Ticket'], axis = 1)
sexEncoder = pd.get_dummies(titanic_in['Sex'])
titanic_in = titanic_in.drop('Sex', axis = 1)
titanic_in = titanic_in.join(sexEncoder)
embarkEncoder = pd.get_dummies(titanic_in['Embarked'])
titanic_in = titanic_in.drop('Embarked', axis = 1)
titanic_in = titanic_in.join(embarkEncoder)
titanic_imp = SimpleImputer(missing_values = np.nan, strategy = 'mean') #could also use mean, median, most freq
titanic_in = pd.DataFrame(imp.fit_transform(titanic_in), columns = titanic_in.columns)
titanic_out = logistic_regression.predict(titanic_in)
sub = pd.DataFrame({'PassengerId': titanic_Passengers, 'Survived': titanic_out})
sub.to_csv("submission.csv", index=False)
sub


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
